## You can also do query through Python by calling SQL

Simple Query through Python

In [14]:
import psycopg2
conn = psycopg2.connect(host="localhost",database="phila", user="postgres", password="5424796")
cur = conn.cursor()


cur.execute("SELECT version();")
create_table_query = '''SELECT tractce, countyfp,statefp,num FROM censuspntnum;'''
cur.execute(create_table_query)


rows = cur.fetchall()
for row in rows:
    print(" -------- ", row)

conn.commit()

More complicated query through Python

In [19]:
import psycopg2
conn = psycopg2.connect(host="localhost",database="phila", user="postgres", password="5424796")
cur = conn.cursor()


cur.execute("SELECT version();")
create_table_query = '''SELECT c.tractce, c.countyfp, c.statefp, t.num FROM (SELECT census3652.tractce, count(philadata3652.panoid) as num
                        FROM philadata3652 
                        LEFT JOIN census3652 ON ST_Intersects(census3652.geom, philadata3652.geom)
                        GROUP BY census3652.tractce) t JOIN census3652 c ON c. tractce = t.tractce;'''
cur.execute(create_table_query)


rows = cur.fetchall()
for row in rows:
    print(" -------- ", row)

conn.commit()

 --------  ('014500', '101', '42', 42)
 --------  ('031200', '101', '42', 156)
 --------  ('021800', '101', '42', 99)
 --------  ('009100', '101', '42', 106)
 --------  ('033600', '101', '42', 249)
 --------  ('004202', '101', '42', 142)
 --------  ('021300', '101', '42', 112)
 --------  ('016800', '101', '42', 156)
 --------  ('007000', '101', '42', 64)
 --------  ('029100', '101', '42', 180)
 --------  ('030000', '101', '42', 213)
 --------  ('000804', '101', '42', 28)
 --------  ('980100', '101', '42', 139)
 --------  ('013300', '101', '42', 95)
 --------  ('012204', '101', '42', 98)
 --------  ('012203', '101', '42', 25)
 --------  ('013602', '101', '42', 21)
 --------  ('034502', '101', '42', 165)
 --------  ('000902', '101', '42', 38)
 --------  ('001202', '101', '42', 56)
 --------  ('001201', '101', '42', 94)
 --------  ('002702', '101', '42', 125)
 --------  ('026301', '101', '42', 101)
 --------  ('000100', '101', '42', 123)
 --------  ('000200', '101', '42', 100)
 --------  

In [ ]:
# SELECT c.tractce, c.countyfp, c.statefp, c.geom, t.num FROM (SELECT census3652.tractce, count(philadata3652.panoid) as num
# FROM philadata3652 
# LEFT JOIN census3652 ON ST_Intersects(census3652.geom, philadata3652.geom)
# GROUP BY census3652.tractce) t JOIN census3652 c ON c. tractce = t.tractce;
